In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import pickle
import torch

In [2]:
import sys
import os

# Add the 'project' directory to the path
sys.path.append(os.path.abspath('..'))

from project_code.data.load_data import load_dataframes
from project_code.data.prepare_data_sklearn import get_features_targets
from project_code.data.prepare_data_pytorch import prepare_data_tensors
from project_code.utils.results import get_best_model_file
from project_code.inference.parameters import get_core_parameter_predictions, convert_output_to_parameter_predictions, PARAMETER_COLS

# Loading data and models

In [3]:
datasets_folder = f'../data/processed/'

dataset_of_model = {
    'SRTaxo1NN': 'biologist_no_pub_age',  
    'Taxo1NN': 'biologist_no_pub_age',  
    'RandomForestRegressor': 'final_taxonomy_ecocodes',
    'MultiTaskElasticNet': 'final_taxonomy_ecocodes',
    'MLP': 'final_taxonomy_ecocodes',
    'MLPSC': 'final_taxonomy_ecocodes',
    'DEBNetHC': 'final_taxonomy_ecocodes',
    'DEBNetSC': 'final_taxonomy_ecocodes',
}


## Loading best models

In [4]:
def load_model(model_file, results_folder):
    if model_file[-4:] == '.pkl':
        with open(f"{results_folder}/models/{model_file}", 'rb') as f:
            model = pickle.load(f)
    elif model_file[-4:] == '.pth':
        model = torch.load(f"{results_folder}/models/{model_file}", weights_only=False)
        model.eval()
    return model

In [5]:
best_models = {}
best_models_test_performance_files = {}
for mt in dataset_of_model.keys():  
    results_folder = f'../results/{dataset_of_model[mt]}'
    metric = 'logQ'
    model_file = get_best_model_file(results_folder=results_folder, model_type=mt, metric=metric)
    #print(mt, model_file)
    if model_file is not None:
        best_models[mt] = load_model(model_file, results_folder)
        test_performance_filename =  model_file[:-4] + '.csv'
        best_models_test_performance_files[mt] = os.path.join(results_folder, 'test_performance', test_performance_filename)

print(best_models.keys())

dict_keys(['SRTaxo1NN', 'Taxo1NN', 'MLP'])


## Loading data

In [6]:
all_dfs = {}
all_col_types = {}
all_data = {}
device = torch.device("cpu")
for dataset_name in list(set(dataset_of_model.values())):
    results_folder = f'../results/{dataset_name}'
    dfs, col_types = load_dataframes(dataset_name=dataset_name, data_split='train_test', datasets_folder=datasets_folder)
    all_dfs[dataset_name] = dfs
    all_col_types[dataset_name] = col_types 
    if 'biologist' in dataset_name:
        if 'SRTaxo1NN' in best_models:
            model = best_models['SRTaxo1NN']
        elif 'Taxo1NN' in best_models:
            model = best_models['Taxo1NN']
        else:
            continue
        encoded_dfs = {}
        # Encoded data with trained model encoders
        for split in ('train', 'test'):
            encoded_dfs[split] = model.regressor.encode_data(all_dfs[dataset_name][split])
        all_data[dataset_name] = get_features_targets(data=encoded_dfs, col_types=all_col_types[dataset_name])
    else:
        data_tensors, dataloaders, datasets, scalers = prepare_data_tensors(data=dfs, col_types=col_types,
                                                                            batch_size=1,
                                                                            scaling_type='log_standardize',
                                                                            device=device)
        all_data[dataset_name] = data_tensors
                                                                            

# Visualize predictions

In [7]:
taxonomy_cols = [col for col in all_col_types['final_taxonomy_ecocodes']['input']['all'] if 'class' in col]

hue_series = {
    'metamorphosis': pd.concat([all_dfs['biologist_no_pub_age'][data_split]['metamorphosis'] for data_split in ['train', 'test']]),
    'class': pd.concat([pd.from_dummies(all_dfs['final_taxonomy_ecocodes'][data_split][taxonomy_cols], sep='_') for data_split in ['train', 'test']])['class'],
    'climate': None,
    'habitat': None,
    'migrate': None,
    'food': None
}

hue_orders = {
    'metamorphosis': [False, True],
    'class': None,
    'climate': None,
    'habitat': None,
    'migrate': None,
    'food': None
}


In [8]:
def plot_residuals_df(model_type, plot_kind, data_split, groupby, scale):
    # Get data for the model
    dataset_name = dataset_of_model[model_type]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    model = best_models[model_type]

    # Get predictions:
    X = data[data_split]['input']
    y_true_ps = data[data_split]['output']
    if 'DEBNet' in model_type:
        y_pred_ps = model.predict(torch.tensor(X, dtype=torch.float32))
    elif 'Taxo1NN' in model_type and data_split in ['train', 'val']:
        distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
        y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(distance_matrix)
        y_pred_ps = model.regressor_.apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
    else:
        y_pred_ps = model.predict(data[data_split]['input'])

    if scale == 'model':
        cols_to_plot = col_types['output']['all']
        target_df = pd.DataFrame(y_true_ps, columns=col_types['output']['all'])
        pred_df = pd.DataFrame(y_pred_ps, columns=col_types['output']['all'])
        
    elif scale == 'parameter':
        cols_to_plot = PARAMETER_COLS     
        mask = data[data_split]['mask']       
        target_df = convert_output_to_parameter_predictions(y=y_true_ps, y_true=y_true_ps, mask=mask, col_types=col_types)
        pred_df = convert_output_to_parameter_predictions(y=y_pred_ps, y_true=y_true_ps, mask=mask, col_types=col_types)

    target_df.set_index(all_dfs[dataset_name][data_split].index, inplace=True)
    pred_df.set_index(all_dfs[dataset_name][data_split].index, inplace=True)
    if plot_kind == 'residual_vs_predicted':
        if scale == 'model':
            residuals_df = target_df - pred_df
        elif scale == 'parameter':
            residuals_df = (target_df - pred_df) / target_df  

    # Plot predictions vs targets  
    n_cols = 3
    n_rows = np.ceil(len(cols_to_plot) / n_cols).astype(int)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(16, 5*n_rows), tight_layout=True)
    fig.suptitle(model_type, fontsize=16)
    margin_factor = 0.05
    for i, col in enumerate(cols_to_plot):
        ax = axes[i // n_cols, i % n_cols]

        if scale == 'parameter' and col != 'kap':
            plot_log_scale_x = True
            if plot_kind == 'residual_vs_predicted':
                plot_log_scale_y = False
            else:
                plot_log_scale_y = True
        elif scale == 'model' and (col in col_types['output']['log'] or 'Taxo1NN' in model_type):
            plot_log_scale_x = True
            plot_log_scale_y = True
        else:
            plot_log_scale_x = False
            plot_log_scale_y = False
        
        if plot_kind == 'residual_vs_predicted':
            sns.scatterplot(x=pred_df[col], y=residuals_df[col], ax=ax, hue=hue_series[groupby], hue_order=hue_orders[groupby]) # Fix
            min_v = (1-margin_factor)*min(target_df[col].min(), pred_df[col].min())
            max_v = (1+margin_factor)*max(target_df[col].max(), pred_df[col].max())
            ax.set_xlim([min_v, max_v])

            ax.plot([min_v, max_v], [0, 0], 'k--')
            ax.set_ylabel('Residuals (actual - predicted)')

        elif plot_kind == 'actual_vs_predicted':
            sns.scatterplot(x=pred_df[col], y=target_df[col], ax=ax, hue=hue_series[groupby], hue_order=hue_orders[groupby])
            min_v = (1-margin_factor)*min(target_df[col].min(), pred_df[col].min())
            max_v = (1+margin_factor)*max(target_df[col].max(), pred_df[col].max())
            ax.set_xlim([min_v, max_v])
            ax.set_ylim([min_v, max_v])
            ax.plot([min_v, max_v], [min_v, max_v], 'k--')
            ax.set_ylabel('Actual values')

        if plot_log_scale_x:
            ax.set_xscale('log')
        else:
            ax.set_xscale('linear')
        
        if plot_log_scale_y:
            ax.set_yscale('log')
        else:
            ax.set_yscale('linear')

        ax.set_xlabel('Predicted values') 
        #r2 = metrics.r2_score(target_df, pred_df)
        ax.set_title(f"{col}")

model_selector = widgets.Dropdown(options=list(best_models.keys()), value='MLP', description='Model:')
plot_selector = widgets.Dropdown(options=['actual_vs_predicted', 'residual_vs_predicted'], value='actual_vs_predicted', description='Plot Type:')
data_split_selector = widgets.Dropdown(options=['train', 'test'], value='test', description='Data Split: ')
groupby_selector = widgets.Dropdown(options=['metamorphosis', 'class', 'climate', 'habitat', 'migrate', 'food'])
scale_selector = widgets.Dropdown(options=['model', 'parameter',], value='parameter')
widgets.interactive(plot_residuals_df, model_type=model_selector, plot_kind=plot_selector, data_split=data_split_selector, groupby=groupby_selector, scale=scale_selector)

interactive(children=(Dropdown(description='Model:', index=2, options=('SRTaxo1NN', 'Taxo1NN', 'MLP'), value='…

# Save parameter predictions

### AmP values

In [9]:
dataset_name = 'biologist_no_pub_age'
dfs = all_dfs[dataset_name]
col_types = all_col_types[dataset_name]
#gt_df = pd.concat({ds: dfs[ds][col_types['output']['all']] for ds in ('train', 'test')}).reset_index(level=0, names='data_split')
#gt_pars_df = get_core_parameter_predictions(dfs, pred_df=gt_df)
#gt_pars_df.to_csv(f'../results/parameter_predictions/AmP_predictions.csv', float_format='%.6e')
#gt_pars_df

### ML models

In [10]:
def save_parameter_predictions_old(model_type):
    model = best_models[model_type]
    dataset_name = dataset_of_model[model_type]
    dfs = all_dfs[dataset_name]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    pred_df = pd.DataFrame()
    for split in ('train', 'test'):
        if 'Taxo1NN' in model_type and split == 'train':
            train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
            y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
            y_pred = model.regressor_.apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
        else:
            y_pred = model.predict(data[split]['input'])
        split_pred_df = pd.DataFrame(data=y_pred, index=dfs[split].index, columns=col_types['output']['all'])
        split_pred_df['data_split'] = split
        pred_df = pd.concat([pred_df, split_pred_df])
    pars_df = get_core_parameter_predictions(dfs, pred_df=pred_df, col_types=col_types)
    predictions_file_name = f'../results/parameter_predictions/{model_type}_predictions.csv'
    pars_df.to_csv(predictions_file_name, float_format='%.10e')
    print(f'Saved parameter predictions for model {model_type} in {predictions_file_name}')

    return pars_df


def save_parameter_predictions(model_type):
    model = best_models[model_type]
    dataset_name = dataset_of_model[model_type]
    dfs = all_dfs[dataset_name]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    pred_df = pd.DataFrame()
    for split in ('train', 'test'):
        if 'Taxo1NN' in model_type and split == 'train':
            train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
            y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
            y_pred = model.regressor_.apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
        else:
            y_pred = model.predict(data[split]['input'])
        split_pred_df = convert_output_to_parameter_predictions(y=y_pred, y_true=data[split]['output'], mask=data[split]['mask'], col_types=col_types)
        split_pred_df.set_index(dfs[split].index, inplace=True)
        split_pred_df['data_split'] = split
        pred_df = pd.concat([pred_df, split_pred_df])
    pars_df = get_core_parameter_predictions(dfs, pred_df=pred_df)
    predictions_file_name = f'../results/parameter_predictions/{model_type}_predictions.csv'
    pars_df.to_csv(predictions_file_name, float_format='%.10e')
    print(f'Saved parameter predictions for model {model_type} in {predictions_file_name}')

    return pars_df

In [11]:
for model_type in best_models:
    save_parameter_predictions(model_type)

Saved parameter predictions for model SRTaxo1NN in ../results/parameter_predictions/SRTaxo1NN_predictions.csv
Saved parameter predictions for model Taxo1NN in ../results/parameter_predictions/Taxo1NN_predictions.csv
Saved parameter predictions for model MLP in ../results/parameter_predictions/MLP_predictions.csv


# Compare parameter predictions

## Test performance

In [12]:
model_list = best_models.keys()
logQ_test_perf_df = pd.DataFrame(index=model_list, columns=PARAMETER_COLS)
smape_test_perf_df = pd.DataFrame(index=model_list, columns=PARAMETER_COLS)
for model_type in model_list:
    model_test_df = pd.read_csv(f"{best_models_test_performance_files[model_type]}", index_col=0)
    logQ_test_perf_df.loc[model_type, :] = model_test_df.loc[:, 'log_accuracy_ratio']
    smape_test_perf_df.loc[model_type, :] = model_test_df.loc[:, 'symmetric_mean_absolute_percentage_error']

In [13]:
logQ_test_perf_df

,p_Am,kap,v,p_M,E_Hb,E_Hj,E_Hp,k_J,s_M
SRTaxo1NN,1.091561,0.158283,0.496554,1.26274,1.655532,2.772625,1.385389,1.13844,0.588246
Taxo1NN,1.123633,0.155328,0.483032,1.214617,0.845133,2.841349,1.435638,1.121036,0.663127
MLP,0.672437,0.096931,0.37317,0.828903,0.974915,2.054981,0.877042,0.84983,0.53738


In [14]:
smape_test_perf_df

,p_Am,kap,v,p_M,E_Hb,E_Hj,E_Hp,k_J,s_M
SRTaxo1NN,39.071745,7.738836,21.814879,41.05639,49.595093,67.343778,48.05035,40.453595,26.620275
Taxo1NN,40.825543,7.590735,21.297077,40.701057,23.388019,67.631117,46.684678,39.873817,28.891509
MLP,28.702351,4.783999,17.25151,33.742659,39.039058,60.279702,36.119939,32.473531,24.814101


In [15]:
model_labels = {
    'DEBNetHC': 'DEBNet w/ HC',
    'DEBNetSC': 'DEBNet w/ SC',
    'Taxo1NN': 'Taxonomic 1-NN',
    'SRTaxo1NN': 'Taxonomic 1-NN w/ S.R.',
    'MultiTaskElasticNet': 'Elastic Net',
    'RandomForestRegressor': 'Random Forest',
    'MLP': 'MLP',
    'MLPSC': 'MLP w/ SC',
}

In [16]:
#for mt, row in mape_df.iterrows():
test_metrics_df_dict = {
    'logQ': logQ_test_perf_df,
    'sMAPE': smape_test_perf_df,
}
for metric, df in test_metrics_df_dict.items():
    print(f'\n\n{metric} test performance:\n')
    for mt in model_list:
        row = df.loc[mt]
        line = f"{model_labels[mt]} "
        for par in PARAMETER_COLS:
            mape = row[par]
            if mape == min(df[par]):
                line += f' & \\textbf{{ {mape:.4f} }}'
            else:
                line += f' & {mape:.4f}'
        line += ' \\\\'
        print(line)



logQ test performance:

Taxonomic 1-NN w/ S.R.  & 1.0916 & 0.1583 & 0.4966 & 1.2627 & 1.6555 & 2.7726 & 1.3854 & 1.1384 & 0.5882 \\
Taxonomic 1-NN  & 1.1236 & 0.1553 & 0.4830 & 1.2146 & \textbf{ 0.8451 } & 2.8413 & 1.4356 & 1.1210 & 0.6631 \\
MLP  & \textbf{ 0.6724 } & \textbf{ 0.0969 } & \textbf{ 0.3732 } & \textbf{ 0.8289 } & 0.9749 & \textbf{ 2.0550 } & \textbf{ 0.8770 } & \textbf{ 0.8498 } & \textbf{ 0.5374 } \\


sMAPE test performance:

Taxonomic 1-NN w/ S.R.  & 39.0717 & 7.7388 & 21.8149 & 41.0564 & 49.5951 & 67.3438 & 48.0503 & 40.4536 & 26.6203 \\
Taxonomic 1-NN  & 40.8255 & 7.5907 & 21.2971 & 40.7011 & \textbf{ 23.3880 } & 67.6311 & 46.6847 & 39.8738 & 28.8915 \\
MLP  & \textbf{ 28.7024 } & \textbf{ 4.7840 } & \textbf{ 17.2515 } & \textbf{ 33.7427 } & 39.0391 & \textbf{ 60.2797 } & \textbf{ 36.1199 } & \textbf{ 32.4735 } & \textbf{ 24.8141 } \\


In [17]:
model_type = 'Taxo1NN'
model = best_models[model_type]
dataset_name = dataset_of_model[model_type]
dfs = all_dfs[dataset_name]
data = all_data[dataset_name]
col_types = all_col_types[dataset_name]
pred_df = pd.DataFrame()
for split in ('train', 'test'):
    if 'Taxo1NN' in model_type and split == 'train':
        train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
        y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
        y_pred = model.regressor_.apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
    else:
        y_pred = model.predict(data[split]['input'])
    split_pred_df = convert_output_to_parameter_predictions(y=y_pred, y_true=data[split]['output'], mask=data[split]['mask'], col_types=col_types)
    split_pred_df.set_index(dfs[split].index, inplace=True)
    split_pred_df['data_split'] = split
    pred_df = pd.concat([pred_df, split_pred_df])
pars_df = get_core_parameter_predictions(dfs, pred_df=pred_df)

In [18]:
pars_df.loc['Amia_calva']

data_split          train
z                6.185801
p_M            164.765016
kap              0.567381
v                0.025085
E_G           5230.125523
E_Hb             4.969683
E_Hx             5.218168
E_Hj             4.969683
E_Hp          1358731.911
k_J                 0.002
Name: Amia_calva, dtype: object

In [19]:
dfs['train'].loc['Amia_calva']

estim_kap                  True
v                      0.025085
family                  Amiidae
estim_p_M                  True
E_Hp                1358731.911
E_Hj                   4.969683
p_M                  164.765016
Wwi                      3800.0
d_V                         0.2
k_J                       0.002
estim_s_M                 False
s_p_M                  0.091386
E_Hb                   4.969683
class            Actinopterygii
estim_v                    True
s_M                         1.0
estim_s_p_M                True
estim_E_Hb                 True
order                Amiiformes
metamorphosis             False
estim_E_Hj                False
estim_E_Hp                 True
phylum                 Chordata
kap                    0.567381
estim_k_J                 False
estim_p_Am                 True
genus                      Amia
Name: Amia_calva, dtype: object